# ICP03-DataPrep
# Rodgers Okeyo Ochieng

Description: Your boss comes to your desk. She praises you for your work on the data preparation of the Air BNB data; but she disagrees with your choice to encode room_type, bed_type, and cancelation_policy as ordinal. She would like to see these encoded using One-Hot-Encoding.

For this assignment, you will begin with a copy of the Airbnb data prep notebook covered in the tutorial. Make only the necessary edits to make the change requested by your boss. Submit your new notebook here.

## Import Libraries

In [1]:
# import numpy and pandas libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.impute import SimpleImputer


# set random seed to ensure that results are repeatable
np.random.seed(8085)

## Load data 

In [2]:
airbnb = pd.read_csv("airbnb.csv")
airbnb.head(3)

,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,...,guests_included,price_per_extra_person,minimum_nights,number_of_reviews,number_days_btw_first_last_review,review_scores_rating,cancellation_policy,price,price_gte_150,price_category
0,0,0,Roslindale,42.282619,-71.133068,House,Entire home/apt,4,1.5,2.0,...,1,0,2,0,0,NaN,moderate,250,1,gte_226
1,0,1,Roslindale,42.286241,-71.134374,Apartment,Private room,2,1.0,1.0,...,0,0,2,36,804,94.0,moderate,65,0,lte_$75
2,1,1,Roslindale,42.292438,-71.135765,Apartment,Private room,2,1.0,1.0,...,1,20,3,41,2574,98.0,moderate,65,0,lte_$75


## Conduct initial exploration of the data

In [3]:
# look at the data
airbnb.head(3) 

,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,...,guests_included,price_per_extra_person,minimum_nights,number_of_reviews,number_days_btw_first_last_review,review_scores_rating,cancellation_policy,price,price_gte_150,price_category
0,0,0,Roslindale,42.282619,-71.133068,House,Entire home/apt,4,1.5,2.0,...,1,0,2,0,0,NaN,moderate,250,1,gte_226
1,0,1,Roslindale,42.286241,-71.134374,Apartment,Private room,2,1.0,1.0,...,0,0,2,36,804,94.0,moderate,65,0,lte_$75
2,1,1,Roslindale,42.292438,-71.135765,Apartment,Private room,2,1.0,1.0,...,1,20,3,41,2574,98.0,moderate,65,0,lte_$75


In [4]:
# generate a basic summary of the data
airbnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3555 entries, 0 to 3554
Data columns (total 23 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   host_is_superhost                  3555 non-null   int64  
 1   host_identity_verified             3555 non-null   int64  
 2   neighbourhood_cleansed             3555 non-null   object 
 3   latitude                           3555 non-null   float64
 4   longitude                          3555 non-null   float64
 5   property_type                      3552 non-null   object 
 6   room_type                          3555 non-null   object 
 7   accommodates                       3555 non-null   int64  
 8   bathrooms                          3541 non-null   float64
 9   bedrooms                           3545 non-null   float64
 10  beds                               3546 non-null   float64
 11  bed_type                           3555 non-null   objec

In [5]:
# generate a statistical summary of the numeric value in the data
airbnb.describe()

,host_is_superhost,host_identity_verified,latitude,longitude,accommodates,bathrooms,bedrooms,beds,Number of amenities,guests_included,price_per_extra_person,minimum_nights,number_of_reviews,number_days_btw_first_last_review,review_scores_rating,price,price_gte_150
count,3555.000000,3555.000000,3555.000000,3555.000000,3555.000000,3541.000000,3545.000000,3546.000000,3555.00000,3555.000000,3555.000000,3555.000000,3555.000000,3555.000000,2755.000000,3555.000000,3555.000000
mean,0.113080,0.727989,42.339973,-71.084874,3.023629,1.215899,1.246544,1.597293,14.85879,1.427004,10.886639,3.116737,19.126582,279.052602,91.891470,166.060478,0.500422
std,0.316735,0.445058,0.024464,0.031614,1.754808,0.492656,0.738440,0.995467,4.82126,1.050204,19.092755,8.273949,35.666178,408.686952,9.548381,103.378456,0.500070
min,0.000000,0.000000,42.235942,-71.171789,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,1.000000,0.000000,0.000000,20.000000,10.000000,0.000000
25%,0.000000,0.000000,42.329875,-71.105183,2.000000,1.000000,1.000000,1.000000,12.00000,1.000000,0.000000,1.000000,1.000000,0.000000,89.000000,85.000000,0.000000
50%,0.000000,1.000000,42.345191,-71.078487,2.000000,1.000000,1.000000,1.000000,15.00000,1.000000,0.000000,2.000000,5.000000,92.000000,94.000000,150.000000,1.000000
75%,0.000000,1.000000,42.354672,-71.062142,4.000000,1.000000,2.000000,2.000000,18.00000,1.000000,20.000000,3.000000,21.000000,402.000000,98.000000,219.000000,1.000000
max,1.000000,1.000000,42.389982,-71.000100,16.000000,6.000000,5.000000,16.000000,30.00000,14.000000,200.000000,300.000000,404.000000,2680.000000,100.000000,650.000000,1.000000


In [6]:
# Check the missing values by summing the total na's for each variable
airbnb.isna().sum()

host_is_superhost                      0
host_identity_verified                 0
neighbourhood_cleansed                 0
latitude                               0
longitude                              0
property_type                          3
room_type                              0
accommodates                           0
bathrooms                             14
bedrooms                              10
beds                                   9
bed_type                               0
Number of amenities                    0
guests_included                        0
price_per_extra_person                 0
minimum_nights                         0
number_of_reviews                      0
number_days_btw_first_last_review      0
review_scores_rating                 800
cancellation_policy                    0
price                                  0
price_gte_150                          0
price_category                         0
dtype: int64

In [7]:
# create a list of the catagorical variables
category_var_list = list(airbnb.select_dtypes(include='object').columns) 
category_var_list

['neighbourhood_cleansed',
 'property_type',
 'room_type',
 'bed_type',
 'cancellation_policy',
 'price_category']

In [8]:
# explore the categorical variable values that are currently represented as string - often there are typos here that need to be fixed.
for cat in category_var_list: # generally, we want to avoid for loops and use a functional style (i.e. list comprehension)
    print(f"{cat}: {airbnb[cat].unique()}\n")

neighbourhood_cleansed: ['Roslindale' 'Jamaica Plain' 'Mission Hill' 'Longwood Medical Area'
 'Bay Village' 'Leather District' 'Chinatown' 'North End' 'Roxbury'
 'South End' 'Back Bay' 'East Boston' 'Charlestown' 'West End'
 'Beacon Hill' 'Downtown' 'Fenway' 'Brighton' 'West Roxbury' 'Hyde Park'
 'Mattapan' 'Dorchester' 'South Boston Waterfront' 'South Boston'
 'Allston']

property_type: ['House' 'Apartment' 'Condominium' 'Villa' 'Bed & Breakfast' 'Townhouse'
 'Entire Floor' 'Loft' 'Guesthouse' 'Boat' 'Dorm' 'Other' nan 'Camper/RV']

room_type: ['Entire home/apt' 'Private room' 'Shared room']

bed_type: ['Real Bed' 'Pull-out Sofa' 'Futon' 'Airbed' 'Couch']

cancellation_policy: ['moderate' 'flexible' 'strict' 'super_strict_30']

price_category: ['gte_226' 'lte_$75' 'btw_$75-$150' 'btw_$151-$225']



### Summary the findings from our initial evaluation of the data

* We have 6 categorical variables
* We have 3 variables that have missing values
* There doesn't seem to be a problem with the catogorical class names.

## Clean and Transform the Data

When clearning and preparing data, we want to make sure we don't introduce information from the test set into the training set. This is a common mistake that can lead to overfitting, and is often referred to as "data leakage", or "information leakage".

To avoid this, we will do the following:
    * Conduct any data prepartion that should be done *BEFORE* the data split.
    * Split the data.
    * Conduct any data preparation that should be done *AFTER* the data split.

### 4.1  Conduct any data prepartion that should be done *BEFORE* the data split

Tasks at this stage include:
1. Drop any columns/features 
2. Decide if you with to exclude any observations (rows) due to missing na's.
2. Conduct proper encoding of categorical variables
    1. You can transform them using dummy variable encoding, one-hot-encoding, or label encoding. 

#### Drop any columns/variables we will not be using

In [9]:
# Our target is price_gtre_150; but there are three related price variables - price, price_gte_150, and price_category. 
# We need to drop price, and price_category; do you know why? Make sure you understand why we are dropping these variables.
airbnb.drop(['price_category', 'price'], axis=1, inplace = True)

#### Drop observations with too many NA's

If we want to remove the rows with NA's use the following code that is commented out. For this exercise - we will not drop rows with NA's 

In [10]:
# If we want to remove rows with NA's use the following code:
# airbnb = airbnb.dropna(axis=0)

In [11]:
# verify that there are now no missing values
# airbnb.isna().sum()

In [12]:
# investigage how many rows remain 
# airbnb.shape

#### Encode our categorical variables

Categorical variables usually have strings for their values. Many machine learning algorithms do not support string values for the input variables. Therefore, we need to replace these string values with numbers. This process is called categorical variable encoding.

In a previous step we identified 5 catagorical variables and found no indication of typos in the class names. Our focus is now on encoding the variables. 

We have three main approaches to encoding variables (these will be discussed in greater detail in class)
* One-Hot-Encoding
* Dummy Encoding
* Label/Ordinal Encoding

In this exercise; we will dummy encode neighbourhood_cleansed, property_type using dummy encoding, and room_type, bed_type and cancelation policy using label encoding. (we will have more discussion on these choices in class).

Before we do our encoding, we must identify if any of our categorical variables have a missing value. We will replace any missing values with the term 'unkown'.

In [13]:
airbnb['property_type'].isna().sum() # check for missing values in this variable/column - we can see there are three for this variable

3

In [14]:
airbnb["property_type"].fillna("unkown", inplace = True)

In [15]:
airbnb['neighbourhood_cleansed'].isna().sum() # check for missing values in this variable/column - we can see there are three for this variable

0

In [16]:
airbnb['room_type'].isna().sum() # based on these results, we can see that there are no missing values

0

In [17]:
airbnb['bed_type'].isna().sum() # based on these results, we can see that there are no missing values

0

In [18]:
airbnb['cancellation_policy'].isna().sum() # based on these results, we can see that there are no missing values

0

Now, let's encode neighborhood_cleansed and property_type as dummy variables and room_type, bed_type and cancelation_policy using one hot encoding

In [19]:
dummies_df = pd.get_dummies(airbnb['neighbourhood_cleansed'], prefix='neighbourhood_cleansed', drop_first=True)

In [20]:
airbnb = airbnb.join(dummies_df)
airbnb.drop('neighbourhood_cleansed', axis=1, inplace = True)

In [21]:
airbnb = airbnb.join(pd.get_dummies(airbnb['property_type'], prefix='property_type', drop_first=True))
airbnb.drop('property_type', axis=1, inplace = True)

# Using the OneHotEncoder

In [22]:
enc = OneHotEncoder(handle_unknown='ignore')
encoded_df = enc.fit_transform(airbnb[['room_type', 'bed_type', 'cancellation_policy']])

In [30]:
# explore the dataframe columns to verify encoding and dropped columns
airbnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3555 entries, 0 to 3554
Data columns (total 56 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   host_is_superhost                               3555 non-null   int64  
 1   host_identity_verified                          3555 non-null   int64  
 2   latitude                                        3555 non-null   float64
 3   longitude                                       3555 non-null   float64
 4   room_type                                       3555 non-null   object 
 5   accommodates                                    3555 non-null   int64  
 6   bathrooms                                       3541 non-null   float64
 7   bedrooms                                        3545 non-null   float64
 8   beds                                            3546 non-null   float64
 9   bed_type                                 

### 4.2 Split data (train/test)

In [31]:
# split the data into validation and training set
train_df, test_df = train_test_split(airbnb, test_size=0.3)

# to reduce repetition in later code, create variables to represent the columns
# that are our predictors and target
target = 'price_gte_150'
predictors = list(airbnb.columns)
predictors.remove(target)

### 4.3  Conduct any data prepartion that should be done *AFTER* the data split

We will look at the following:
1) imput any missing numeric values using the mean of the variable/column
2) remove differences of scale by standardizing the numerica variables

#### Impute missing values

In [32]:
numeric_cols_with_nas = list(train_df.isna().sum()[train_df.isna().sum() > 0].index)
numeric_cols_with_nas

['bathrooms', 'bedrooms', 'beds', 'review_scores_rating']

We can see from the code above that there are 4 variables (columns) that contain missing numeric values (we've already taken care of any missing values in the catagorical variables earlier).

In [33]:
imputer = SimpleImputer(strategy="median")

train_df[numeric_cols_with_nas] = imputer.fit_transform(train_df[numeric_cols_with_nas])
test_df[numeric_cols_with_nas] = imputer.transform(test_df[numeric_cols_with_nas])

#### Standardize numeric values

Now, let's create a common scale between the numberic columns by standardizing each numeric column

In [34]:
# create a standard scaler and fit it to the training set of predictors
scaler = preprocessing.StandardScaler()
cols_to_stdize = ['latitude', 'longitude', 'accommodates', 
                   'bathrooms', 'bedrooms', 'beds', 'Number of amenities', 
                   'guests_included', 'price_per_extra_person', 'minimum_nights', 
                   'number_of_reviews', 'number_days_btw_first_last_review', 
                   'review_scores_rating']                
               
# Transform the predictors of training and validation sets
train_df[cols_to_stdize] = scaler.fit_transform(train_df[cols_to_stdize]) # train_predictors is not a numpy array


test_df[cols_to_stdize] = scaler.transform(test_df[cols_to_stdize]) # validation_target is now a series object


## 5.0 Save the data

In [35]:
train_X = train_df[predictors]
train_y = train_df[target] # train_target is now a series objecttrain_df.to_csv('airbnb_train_df.csv', index=False)
test_X = test_df[predictors]
test_y = test_df[target] # validation_target is now a series object

train_df.to_csv('airbnb-train-price_gte_150.csv', index=False)
test_df.to_csv('airbnb-test-price_gte_150.csv', index=False)
